In [1]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np

true = np.asarray([[3, 2, 1, 0, 0]])
relevance = np.asarray([[3, 2, 0, 0, 1]])

print(ndcg_score(true, relevance))

#0.980840401274087

0.980840401274087


In [2]:
# 4.5
# Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

# Реальные оценки: [2, 4, 1, 1, 1]
# Предсказанные оценки: [2, 5, 2, 3, 1]
# Вычислите коэффициент NDCG. Округлите результат до двух знаков после точки-разделителя.

true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])

print(ndcg_score(true, relevance))

0.9748317848747132


## 5. Проблема холодного старта и popularity-based model

In [3]:
# Мы будем работать с набором данных Movie Lens. 
# Он содержит идентификаторы для каждого фильма и пользователя, который его смотрел, а также оценку, 
# которую пользователь поставил фильму. 
# 
# В датасете представлено 25 000 095 оценок фильмов от 162 541 пользователя со шкалой оценок от 0.5 до 5.0.
import pandas as pd

ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv('data/movie.csv')
df=pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [4]:
# 5.1
# Подсчитайте, сколько раз каждый фильм встречается в наборе данных. 
# Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

df['title'].value_counts()

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
                                    ...  
Richard III (1912)                      1
The Nautical Chart (2007)               1
Two Arabian Knights (1927)              1
Night Flight (1933)                     1
Série noire (1979)                      1
Name: title, Length: 26729, dtype: int64

In [12]:
# 5.2
# Найдите средний рейтинг для каждого из фильмов.
# Найдите фильмы с наивысшим средним рейтингом.
# Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, 
# если предварительно отсортировать их по алфавитному порядку.
# Впишите только название фильма, без кавычек и без года выхода на экран.

# Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

df_group = df.groupby('title')['rating'].mean().sort_values()
mask = df_group==5.0
df_group[mask].sort_index()

title
1971 (2014)                                       5.0
A Blank on the Map (1971)                         5.0
A Gun for Jennifer (1997)                         5.0
A Night for Dying Tigers (2010)                   5.0
Abendland (2011)                                  5.0
                                                 ... 
Welcome to Australia (1999)                       5.0
When I Walk (2013)                                5.0
Who Killed Vincent Chin? (1987)                   5.0
Year Zero: The Silent Death of Cambodia (1979)    5.0
Yonkers Joe (2008)                                5.0
Name: rating, Length: 113, dtype: float64

In [41]:
# 5.3
# В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. 
# Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, 
# насколько фильм нравится аудитории, 
# а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.


# Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.
# Сколько таких фильмов?

counts = df['title'].value_counts()
mask = counts.values>50
films = list(counts[mask].index)
counts[mask]


Pulp Fiction (1994)                             67310
Forrest Gump (1994)                             66172
Shawshank Redemption, The (1994)                63366
Silence of the Lambs, The (1991)                63299
Jurassic Park (1993)                            59715
                                                ...  
Serpent's Egg, The (Schlangenei, Das) (1977)       51
Lion of the Desert (1980)                          51
Lakeboat (2000)                                    51
Mad Love (1935)                                    51
Kid Brother, The (1927)                            51
Name: title, Length: 10472, dtype: int64

In [45]:
# 5.4
# Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, 
# и найдите среди них фильм с наивысшей оценкой. 
# В качестве ответа запишите название этого фильма без артикля и года выхода на экран.

mask = df['title'].isin(films)
df[mask].groupby('title')['rating'].mean().sort_values()


title
SuperBabies: Baby Geniuses 2 (2004)    0.837321
From Justin to Kelly (2003)            0.973005
Bratz: The Movie (2007)                1.105556
Glitter (2001)                         1.124088
Barney's Great Adventure (1998)        1.163484
                                         ...   
Godfather: Part II, The (1974)         4.275641
Schindler's List (1993)                4.310175
Usual Suspects, The (1995)             4.334372
Godfather, The (1972)                  4.364732
Shawshank Redemption, The (1994)       4.446990
Name: rating, Length: 10472, dtype: float64

## 6. Практика

In [48]:
# Начнём работать с файлом shared_articles.csv. Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

# 6.1
# Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. 
# Сколько таких объектов в получившейся таблице?

shared = pd.read_csv('data/shared_articles.csv')
mask = shared['eventType']== 'CONTENT SHARED'
shared=shared[mask]
shared

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1487946604,CONTENT SHARED,9213260650272029784,3609194402293569455,7144190892417579456,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
3118,1487947067,CONTENT SHARED,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
3119,1488223224,CONTENT SHARED,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
3120,1488300719,CONTENT SHARED,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [49]:
#  у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. 
# Составим его из реакций пользователей, придав им разные веса:

event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [52]:
# 6.2
# Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). 
# Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

users = pd.read_csv('data/users_interactions.csv')
users['v_event'] = users['eventType'].apply(lambda x: event_type[x])
users['v_event'].mean()

1.2362885828078327

In [57]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,v_event
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [78]:
# 6.3
# Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, 
# которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

group = users.groupby(by='personId',as_index=False)['contentId'].value_counts()
person = group.groupby('personId')['contentId'].count().sort_values()
mask = person.values>=5
person_list = list(person[mask].index)
person[mask]


personId
-5179891559567091709      5
 7888432621799586316      5
-5444946123447491034      5
 202369545176637828       5
-3381527457560577265      5
                       ... 
-2979881261169775358    437
-1443636648652872475    585
-1032019229384696495    648
-2626634673110551643    669
 3609194402293569455    961
Name: contentId, Length: 1140, dtype: int64

In [82]:
# 6.4 
# Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей 
# (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?
mask = users['personId'].isin(person_list)
users = users[mask]
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,v_event
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [83]:
# Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, 
# то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, 
# и всё это отразилось в трёх действиях. 
# Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. 
# Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. 
# Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. 
# В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

import math

def smooth_user_preference(x):
    return math.log(1+x, 2)

In [112]:
# 6.5
# Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. 
# Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

# Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

# Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

data = users.groupby(by=['personId','contentId'], as_index=False)['v_event'].sum()
data['timestamp'] = users.groupby(by=['personId','contentId'], as_index=False)['timestamp'].max()['timestamp'] # добавляем столбец с последней датой
data['v_event'] = data['v_event'].apply(smooth_user_preference) # делаем логорифмирование
data['timestamp'].mean()


1470605340.0403006

In [120]:
# 6.6
# Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. 
# Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?

mask = data['timestamp']>= 1475519545
test = data[mask]
train =data.drop(test.index, axis=0)
train

,personId,contentId,v_event,timestamp
0,-9223121837663643404,-8949113594875411859,1.000000,1462452127
1,-9223121837663643404,-8377626164558006982,1.000000,1473938707
2,-9223121837663643404,-8208801367848627943,1.000000,1469706702
3,-9223121837663643404,-8187220755213888616,1.000000,1467823897
5,-9223121837663643404,-7331393944609614247,1.000000,1463486417
...,...,...,...,...
39087,9210530975708218054,4788854083489560153,1.584963,1466443715
39092,9210530975708218054,6527704900182256036,2.169925,1469621765
39093,9210530975708218054,6940370558848102253,1.000000,1472491975
39095,9210530975708218054,7434270606888620096,3.169925,1468265243


In [143]:
# Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, 
# где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. 
# На место пустых ячеек поместите пустые списки.

final_df = (
    train.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df

,true_train,true_test
personId,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[]
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084]
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]"
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -..."
...,...,...
9165571805999894845,"[-5488842573681626972, -3959242148361340089, -...",[]
9187866633451383747,"[-7660505434580831027, -5571606607344218289, -...","[-9039021554659316935, -4925213312961842798, -..."
9191849144618614467,"[-8742648016180281673, -8208801367848627943, -...","[-5605799891597699962, -4029704725707465084]"


In [125]:
# 6.7
# Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! 
# Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

# Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). 
# Выберите ID самой популярной статьи:

train.groupby('contentId')['v_event'].sum().sort_values(ascending=False)

contentId
-6783772548752091658    231.177195
-133139342397538859     228.024567
-8208801367848627943    189.937683
 8224860111193157980    186.044680
 7507067965574797372    179.094002
                           ...    
 1729892529360390310      1.000000
 1719976830095479814      1.000000
 7933362470375280252      1.000000
 6240076106289531207      1.000000
 9178250635927683         1.000000
Name: v_event, Length: 2366, dtype: float64

In [152]:
# 6.8
# Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). 
# После этого усредните результат по всем пользователям.

recomend_content = list(train.groupby('contentId')['v_event'].sum().sort_values(ascending=False).head(10).index)
final_df['recomend'] = 'l'
for i in final_df.index:
    final_df['recomend'][i] = recomend_content

#Для вычисления precision@10 воспользуйтесь следующей функцией:

def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
    
final_df['precision'] = precision('recomend')
final_df


,true_train,true_test,recomend,precision
personId,,,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[],"[-6783772548752091658, -133139342397538859, -8...",0.004286
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084],"[-6783772548752091658, -133139342397538859, -8...",0.004286
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]","[-6783772548752091658, -133139342397538859, -8...",0.004286
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
...,...,...,...,...
9165571805999894845,"[-5488842573681626972, -3959242148361340089, -...",[],"[-6783772548752091658, -133139342397538859, -8...",0.004286
9187866633451383747,"[-7660505434580831027, -5571606607344218289, -...","[-9039021554659316935, -4925213312961842798, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
9191849144618614467,"[-8742648016180281673, -8208801367848627943, -...","[-5605799891597699962, -4029704725707465084]","[-6783772548752091658, -133139342397538859, -8...",0.004286
